In [1]:
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

def getOlx():
    olx_list = []
    for i in tqdm(range(1,101), desc='OLX'):
        r = requests.get(f'https://www.olx.com.br/imoveis/estado-sp?o={i}', headers=headers)
        content = r.content
        soup = BeautifulSoup(r.content, 'html.parser') 
        li_tags = soup.find_all('li')
        # Iterando sobre as tags <li> encontradas
        for li_tag in li_tags:
            # Verificando se a tag <li> contém um link
            if li_tag.find('a'):
                # Obtendo o valor do atributo "href" do link
                link = li_tag.find('a')['href']        
                if link.startswith('https://sp.olx.com.br'):
                    olx_list.append(link)
    return olx_list

def getLoft():
    temp = []
    loft_list = []
    for i in tqdm(range(1, 133),desc='Loft'):
            response = requests.get(f'https://loft.com.br/venda/imoveis/sp/sao-paulo?utm_source=google&utm_medium=cpc&utm_campaign=ins_01_br_001_sp_0001_sao-paulo_all_aw_search_conversion_broad_&utm_content=all_loft-pura&utm_id=1756086660&utm_placement=&utm_ad_id=576438774862&gclid=Cj0KCQjwjryjBhD0ARIsAMLvnF9KvelK1iKF7xoTJ3HS0KSWMhoDO0UlANc1gTfQEMGg5Bud0HiPZg4aAscOEALw_wcB&pagina={i}')
            html_content = response.text

            soup = BeautifulSoup(html_content, 'html.parser')

            divs = soup.find_all('div', class_='MuiGrid-root')

            for div in divs:
                tag_a = div.find('a')
                if tag_a is not None:
                    link = tag_a['href']
                    if link.startswith('/imovel/'):
                        temp.append(link)
    loft_list = ['http://loft.com.br' + valor for valor in temp]
    return loft_list

## MAIN CODE
olx_list = getOlx()
loft_list = getLoft()

## get olx data


dados_gerais = []  # Lista para armazenar os dados de todas as URLs

for url in tqdm(olx_list):
    r = requests.get(url, headers=headers)
    content = r.content
    soup = BeautifulSoup(r.content, 'lxml') 

    h2 = soup.find('h2', class_='ad__sc-1wimjbb-1 hoHpcC sc-cooIXK cXlgiS')
    if h2:
        h2_value = h2.text.strip()
    else:
        h2_value = ''

    divs = soup.find_all('div', class_='sc-hmzhuo ad__sc-1f2ug0x-3 sSzeX sc-jTzLTM iwtnNi')  # Substitua 'nome_da_classe' pela classe real das divs

    dados_url = {}  # Dicionário para armazenar os dados de uma URL específica

    # Adicione a URL ao dicionário da URL específica com a chave 'URL'
    dados_url['URL'] = url

    # Adicione o valor da tag h2 ao dicionário da URL específica com a chave 'Valor'
    dados_url['Valor'] = h2_value

    # Itere sobre as divs encontradas
    for div in divs:
        # Inicialize um dicionário vazio para armazenar os valores da div atual
        dados = {}

        # Encontre todas as tags dt dentro da div atual
        tags_dt = div.find_all('dt')

        # Itere sobre as tags dt e obtenha os valores
        for dt in tags_dt:
            key = dt.text.strip()  # Use strip() para remover espaços em branco extras

            # Encontre a próxima tag dd
            dd = dt.find_next_sibling('dd')

            # Se a tag dd existir, obtenha o valor
            if dd:
                value = dd.text.strip()
            else:
                # Se não houver uma tag dd, encontre a próxima div com a classe específica
                proxima_div = div.find_next('div', class_='ad__sc-1f2ug0x-2 eSYIff')

                # Encontre a tag a dentro da próxima div
                a = proxima_div.find('a') if proxima_div else None

                # Obtenha o valor da tag a, se existir
                value = a.text.strip() if a else ''

            dados[key] = value

        # Adicione o dicionário da div atual ao dicionário da URL específica
        dados_url.update(dados)

    # Adicione o dicionário da URL específica à lista geral
    dados_gerais.append(dados_url)

# Converter a lista de dicionários em um DataFrame do pandas
df = pd.DataFrame(dados_gerais)

# Salvar o DataFrame em um arquivo Excel
df.to_excel('dados.xlsx', index=False)



Loft: 100%|██████████| 132/132 [00:27<00:00,  4.78it/s]


In [ ]:
## LOFT
import re
import pandas as pd

dados_imoveis = []

for url in tqdm(loft_list):
    r = requests.get(url, headers=headers)
    content = r.content
    soup = BeautifulSoup(r.content, 'html.parser')
    desc = soup.find('h1', class_='MuiTypography-root jss197 jss174 jss183 MuiTypography-body1')
    title = soup.find('title')

    padrao_categoria = r"^(\w+)"
    padrao_tipo = r"à\s(\w+)"
    padrao_bairro = r"em\s(.+?)\scom"
    padrao_tamanho = r"(\d+)\s*m²"
    padrao_quartos = r"(\d+)\squartos"
    padrao_vaga = r"(\d+)\svaga"
    padrao_valor = r"R\$\s([\d.,]+)"

    dados = {}

    try:
        dados['categoria'] = re.search(padrao_categoria, desc.text).group(1)
    except:
        dados['categoria'] = 'Unknow'
    try:
        dados['tipo'] = re.search(padrao_tipo, desc.text).group(1)
    except:
        dados['tipo'] = 'Unknow'
    try:
        dados['bairro'] = re.search(padrao_bairro, desc.text).group(1)
    except:
        dados['bairro'] = 'Unknow'
    try:
        dados['tamanho'] = re.search(padrao_tamanho, desc.text).group(1)
    except:
        dados['tamanho'] = '0'
    try:
        dados['quartos'] = re.search(padrao_quartos, desc.text).group(1)
    except:
        dados['quartos'] = '0'
    try:
        dados['vaga'] = re.search(padrao_vaga, desc.text).group(1)
    except:
        dados['vaga'] = '0'
    try:
        dados['valor'] = re.search(padrao_valor, title.text).group(1)
    except:
        dados['valor'] = '0'

    dados['URL'] = url

    dados_imoveis.append(dados)

# Cria o DataFrame
df = pd.DataFrame(dados_imoveis)

# Define a ordem das colunas
colunas = ['URL', 'Valor', 'CEP', 'Município', 'Bairro', 'Logradouro', 'Categoria', 'Vagas na garagem',
           'Quartos', 'Acomoda', 'Características', 'Tipo', 'Área construída', 'Banheiros',
           'Detalhes do imóvel', 'Condomínio', 'IPTU', 'Área útil', 'Detalhes do condomínio', 'Tamanho']

# Reorganiza as colunas
df = df[colunas]

# Exporta o DataFrame para um arquivo Excel
df.to_excel('dados_imoveis.xlsx', index=False)


In [68]:
## ZAP IMOVEIS
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
import re
import pandas as pd

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
data = []

for i in tqdm(range(1, 101)):
    r = requests.get(f'https://www.zapimoveis.com.br/venda/apartamentos/?pagina={i}', headers=headers)
    content = r.content
    soup = BeautifulSoup(r.content, 'html.parser') 

    price_p = soup.find_all('p', class_='simple-card__price js-price color-darker heading-regular heading-regular__bolder align-left')
    size_value = soup.find_all('span', itemprop='floorSize')
    rooms_value = soup.find_all('span', itemprop='numberOfRooms')
    parking_value = soup.find_all('li', class_='feature__item text-small js-parking-spaces')
    bathroom_value = soup.find_all('span', itemprop='numberOfBathroomsTotal')
    address_value = soup.find_all('h2', class_='simple-card__address color-dark text-regular')
    neighborhood_value = soup.find_all('h2', class_='simple-card__address color-dark text-regular')

    for price, size, rooms, parking, bathroom, address, neighborhood in zip(price_p, size_value, rooms_value, parking_value, bathroom_value, address_value, neighborhood_value):
        temp = {}  # Criar um novo dicionário temp em cada iteração

        apt_value = price.find('strong')    
        temp['price'] = apt_value.text.strip()
        temp['size'] = size.text.strip()
        temp['rooms'] = rooms.text.strip()
        
        try:
            temp['parking'] = re.search(r'\d+', parking.text).group()
        except AttributeError:
            temp['parking'] = '0'  # If parking info is not found, set it as 'N/A'
        
        try:
            temp['bathroom'] = bathroom.text.strip()
        except AttributeError:
            temp['bathroom'] = '0'  # If bathroom info is not found, set it as 'N/A'
        
        temp['address'] = re.sub(r',.*', '',address.text).strip()
        temp['neighborhood'] = re.sub(r'.*,', '', neighborhood.text).strip()
        
        data.append(temp)

df = pd.DataFrame(data)
df.to_excel('data-zap.xlsx', index=False)


100%|██████████| 100/100 [05:02<00:00,  3.02s/it]


In [48]:
## vivareal
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
import re
import pandas as pd

data = []


headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
for i in tqdm(range(1, 279)):
    r = requests.get(f'https://www.vivareal.com.br/venda/sp/sao-paulo/?pagina={i}', headers=headers)
    content = r.content
    soup = BeautifulSoup(r.content, 'html.parser') 
    url_ad = soup.find_all('div', class_='property-card__main-link')
    type_ad = soup.find_all('span', class_='property-card__title js-cardLink js-card-title')
    price = soup.find_all('div', class_='property-card__price js-property-card-prices js-property-card__price-small')
    neighborhood_value = soup.find_all('span', class_='property-card__address')
    measure = soup.find_all('span', class_='property-card__detail-value js-property-card-value property-card__detail-area js-property-card-detail-area')
    rooms = soup.find_all('li', class_='property-card__detail-item property-card__detail-room js-property-detail-rooms')
    bathrooms = soup.find_all('li', class_='property-card__detail-item property-card__detail-bathroom js-property-detail-bathroom')
    garage = soup.find_all('li', class_='property-card__detail-item property-card__detail-garage js-property-detail-garages')

    for url, type_ad, price, neighborhood, measure, room, bathroom, garages in zip(url_ad, type_ad, price, neighborhood_value, measure, rooms, bathrooms, garage):
        temp = {}
        url_link = url.find('a')    
        temp['URL'] = 'https://www.vivareal.com.br/imovel/' + url_link['href']
        temp['Type'] = re.search(r'\w+', type_ad.text).group()
        temp['Price'] = price.text.strip()
        temp['Address'] = neighborhood.text.strip()
        temp['Measure'] = measure.text.strip()
        temp['Rooms'] = room.text.strip()
        temp['bathrooms'] = bathroom.text.strip()
        temp['garage'] = garages.text.strip()
        data.append(temp)

df = pd.DataFrame(data)

df['Rooms'] = df['Rooms'].str.extract('(\d+)')
df['bathrooms'] = df['bathrooms'].str.extract('(\d+)')
df['garage'] = df['garage'].str.extract('(\d+)')
df['Price'] = df['Price'].str.replace('R\$', '')
df['Price'] = df['Price'].str.replace('.', '')
df['Price'] = df['Price'].str.replace(' ', '')
df['garage'] = df['garage'].fillna(0)
df['Type'] = df['Type'].str.capitalize()
df['Address'] = df['Address'].str.replace(', São Paulo.*', '')
df['Address'] = df['Address'].str.replace('.*-', '')
df['Measure'] = df['Measure'].str.replace('-.*', '')
df['Price'] = pd.to_numeric(df['Price'])
df['Measure'] = pd.to_numeric(df['Measure'])
df['Rooms'] = pd.to_numeric(df['Rooms'])
df['bathrooms'] = pd.to_numeric(df['bathrooms'])
df['garage'] = pd.to_numeric(df['garage'])
df.to_excel('data-vivareal.xlsx', index=False)

100%|██████████| 278/278 [04:07<00:00,  1.12it/s]


In [ ]:
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
import re
import pandas as pd

data =[]

for i in tqdm(range(1, 834)):
    r = requests.get(f'https://emcasa.com/imoveis/sp/sao-paulo?pagina={i}')
    content = r.content
    soup = BeautifulSoup(r.content, 'html.parser') 
    url_ad = soup.find_all('a', class_='ListingCard_ecListingContainer__CBKdQ')
    address = soup.find_all('p', class_='Typography_ecTypographyLabel__jXase Typography_ecTypographyRegular__FX_vf')
    prices = soup.find_all('div', class_='ListingCard_ecListingCardPriceAndLabel__ZKCBN')
    measure = soup.find_all('strong', class_='Typography_ecTypographyParagraphSmall__4HQRq Typography_ecTypographyBold__cxADa')
    rooms = soup.find_all('p', class_='Typography_ecTypographyLabel__jXase Typography_ecTypographyRegular__FX_vf')
    garages = soup.find_all('p', class_='Typography_ecTypographyLabel__jXase Typography_ecTypographyRegular__FX_vf')

    temp_adress = []
    temp_measure = []
    temp_rooms = []
    temp_garage = []

    for address in address:
        match = re.search(r'^(.*),\s*São Paulo', address.text)    
        if match:
            result = match.group(1)
            temp_adress.append(result)       
        else:        
            continue

    for measure in measure:
        match = re.search(r'(\d+)\s*m²', measure.text)
        if match:
            result = match.group(1)
            temp_measure.append(result)             
        else:        
            continue

    for room in rooms:
        match = re.search(r'(\d+)\s*(?:quarto|quartos)', room.text)
        
        if match:
            result = match.group(1)
            temp_rooms.append(result)             
        else:        
            continue

    count = 0
    for garage in garages:
        match = re.search(r'(\d+)\s*(?:vaga|vagas)', garage.text)   

        if match:        
            result = match.group(1)        
            temp_garage.append(result)     
            count = 0        
        else:    
            count += 1  
            if count > 5:
                temp_garage.append('0')
                count = 0
                continue

    for url, price, address, measure, room, garage  in zip(url_ad, prices, temp_adress, temp_measure, temp_rooms, temp_garage):
        temp = {}
        temp['URL'] = 'https://emcasa.com' + url['href']
        temp['Price'] = re.search(r'R\$\s*([0-9\.]+),', price.text).group(1)
        temp['Address'] = address
        temp['Measure'] = measure
        temp['Rooms'] = room
        temp['garage'] = garage   
        
        data.append(temp)
        
df = pd.DataFrame(data)
df['Price'] = df['Price'].str.replace('.', '')
df['Price'] = pd.to_numeric(df['Price'])
df['Measure'] = pd.to_numeric(df['Measure'])
df['Rooms'] = pd.to_numeric(df['Rooms'])
df['garage'] = pd.to_numeric(df['garage'])
df['Address'] = df['Address'].str.replace('.*,', '')

df.to_excel('data-emcasa.xlsx', index=False)

